# RAG Evaluation


In [1]:
!uv pip install -q \
    requests==2.32.5 \
    python-dotenv==1.2.1 \
    tqdm==4.67.1 \
    litellm==1.78.5

In [ ]:
import hashlib
import json
import random
import time
import uuid
from collections import defaultdict
from pathlib import Path

import litellm
import requests
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [ ]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]

    return document_id

In [ ]:
for doc in documents:
    doc["id"] = generate_document_id(doc)

In [ ]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [ ]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc["id"]
    hashes[doc_id].append(doc)

len(hashes), len(documents)

(947, 948)

In [ ]:
for k, v in hashes.items():
    if len(v) > 1:
        print(k, len(v))

593f7569 2


In [ ]:
hashes["593f7569"]

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [ ]:
with open("documents-with-ids.json", "wt") as f_out:
    json.dump(documents, f_out, indent=2)

In [11]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [ ]:
prompt_template = """
You are a student who's taking a course.
Formulate 5 questions you might ask based on a FAQ record. The record should
contain the answer to the questions, and the questions should be complete and
not too short.
If possible, use a fewer words as possible from the record.

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", "question3"]
""".strip()

In [18]:
!docker exec -it ollama ollama pull qwen2.5:0.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling c5396e06af29: 100% ▕██████████████████▏ 397 MB                         
pulling 66b9ea09bd5b: 100% ▕██████████████████▏   68 B                         
pulling eb4402837c78: 100% ▕██████████████████▏ 1.5 KB                         
pulling 832dd9e00a68: 100% ▕██████████████████▏  11 KB                         
pulling 005f95c74751: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 


In [25]:
!docker exec -it ollama ollama list

NAME            ID              SIZE      MODIFIED      
qwen2.5:0.5b    a8b0c5157701    397 MB    7 minutes ago    


In [ ]:
response = litellm.completion(
    model="ollama/qwen2.5:0.5b",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain transformers in one paragraph."},
    ],
    api_base="http://localhost:11434",
    api_key="ollama",
    custom_llm_provider="ollama",
)

print(response.choices[0].message["content"])

Transformers are a class of pre-trained neural network models widely used for tasks such as image and speech recognition. They are characterized by their self-attention mechanism, which enables them to process input data not only through the sequence (as seen in word-level processing) but also at different levels of abstraction, resulting in improved downstream performance compared to their predecessors. Transformers build upon a series of components including positional encoding, multi-head attention, and feedforward networks. Their architecture is similar to RNNs or LSTMs with additional layers that maintain state information as they pass through the model. This allows them to handle complex sequence tasks more efficiently than traditional models by enabling them to automatically learn patterns in long sequences without needing explicit token-by-token processing. Additionally, Transformers can be fine-tuned on small datasets at a scale much larger than what was previously feasible, m

In [ ]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = litellm.completion(
        model="ollama/qwen2.5:0.5b",
        messages=[{"role": "user", "content": prompt}],
        api_base="http://localhost:11434",
        api_key="ollama",
        custom_llm_provider="ollama",
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
OUTPUT_PATH = Path("generated_questions.json")

if OUTPUT_PATH.exists():
    with OUTPUT_PATH.open("r", encoding="utf-8") as f:
        results = json.load(f)
else:
    results = {}

In [ ]:
def save_results(data, path=OUTPUT_PATH):
    tmp_path = path.with_suffix(".tmp")
    with tmp_path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    tmp_path.replace(path)

In [ ]:
for doc in tqdm(documents):
    doc_id = str(doc["id"])

    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

    save_results(results)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
len(results)

13